In [1]:
from bs4 import BeautifulSoup 
import requests 

def get_soup_by_link(link):
    response = requests.get(link) 
    return BeautifulSoup(response.text, 'html.parser')
 
def parse_rating(table_movie_rating): 
    try: 
        return table_movie_rating.tbody.tr.td.div.span.string 
    except: 
        return "Unknown" 
    
def parse_genre(table_movie_info): 
    try: 
        return table_movie_info.find('td', {'class': 'genre'}).p.a.string
    except: 
        return "Unknown" 
    
def parse_year(table_movie_info):
    try:
        return table_movie_info.find('td', {'class': 'year'}).a.string
    except:
        return "Unknown"

def parse(table_movie_info, class_name):
    try:
        return table_movie_info.find('td', {'class': class_name}).string
    except:
        return "Unknown"

def parse_min_age(td_with_data):
    try:
        return td_with_data.find('div', {'class': 'title__labels'}).contents[1].string
    except:
        return "Unknown"

def parse_director(td_with_data):
    try:
        return td_with_data.findAll('p', recursive = False)[0].text[10:]
    except:
        return "Unknown"    

def parse_actors(td_with_data):
    try:
        return td_with_data.findAll('p', recursive = False)[1].text[9:]
    except:
        return "Unknown"

main_link = 'https://afisha.tut.by/film/'
soup = get_soup_by_link(main_link)
films_li = soup.findAll('li', {'class': 'lists__li'}) 
links = []
for film_li in films_li: 
    film_links = film_li.findAll('a', {'class': 'name'}) 
    if len(film_links) != 0: 
        link = film_links[0]['href']
        if link.startswith(main_link) and '?utm_source' not in link: 
            links.append(link)

films = {}
links_amount = len(links)
i = 1
for link in links:
    print(str(i) + " of " + str(links_amount) + ": " + link)
    i = i + 1
    
    soup = get_soup_by_link(link)
    td_with_data = soup.find('td', {'class': 'post b-event-post'}) 
    table_movie_info = td_with_data.find('table', {'class': 'movie_info'}) 
    table_movie_rating = td_with_data.find('table', {'class': 'movie_rating'}) 
    name = td_with_data.h1.string
    
    films[name] = {}
    films[name]['link'] = link
    films[name]['genre'] = parse_genre(table_movie_info)
    films[name]['year'] = parse_year(table_movie_info)
    films[name]['country'] = parse(table_movie_info, 'author')
    films[name]['duration'] = parse(table_movie_info, 'duration')
    films[name]['end_date'] = parse(table_movie_info, 'date')
    films[name]['director'] = parse_director(td_with_data) 
    films[name]['actors'] = parse_actors(td_with_data)
    films[name]['rating'] = parse_rating(table_movie_rating)
    films[name]['min_age'] = parse_min_age(td_with_data) 

1 of 94: https://afisha.tut.by/film/malefisenta-vladychica-tmy/
2 of 94: https://afisha.tut.by/film/tekst/
3 of 94: https://afisha.tut.by/film/oni/
4 of 94: https://afisha.tut.by/film/okey-leksi/
5 of 94: https://afisha.tut.by/film/dzhudi-2019/
6 of 94: https://afisha.tut.by/film/proshchanie-2019/
7 of 94: https://afisha.tut.by/film/maleficent_mistress_of_evil_ru_sub/
8 of 94: https://afisha.tut.by/film/zelenee-travy/
9 of 94: https://afisha.tut.by/film/arakhisovyy_sokol_predpremera_s_kinokritikom/
10 of 94: https://afisha.tut.by/film/joker-film/
11 of 94: https://afisha.tut.by/film/gemini/
12 of 94: https://afisha.tut.by/film/everest-film/
13 of 94: https://afisha.tut.by/film/devushki-byvayut-raznye/
14 of 94: https://afisha.tut.by/film/matrica/
15 of 94: https://afisha.tut.by/film/dozhdlivyy-den-v-nyu-yorke/
16 of 94: https://afisha.tut.by/film/once-upon-a-time-in-hollywood/
17 of 94: https://afisha.tut.by/film/k-zvezdam/
18 of 94: https://afisha.tut.by/film/bolevoy-porog/
19 of 94: 

In [10]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd

spark = SparkSession.builder.appName("minald").getOrCreate()
sc = spark.sparkContext
pdDF = pd.DataFrame(films).transpose()
#pdDF
#films

In [8]:
data = [{'a': 1, 'b': 2, 'c':3}, {'a':10, 'b': 20, 'c': 30}] 
movies_df = spark.createDataFrame(pdDF)
movies_df.show()
#movies_df.createOrReplaceTempView("all_movies")
#movies_df.groupBy("min_age").count().show()
#parts = spark.sql("SELECT COUNTRY FROM all_movies")

RecursionError: maximum recursion depth exceeded in comparison

In [11]:
import pyspark.sql.functions as f
countries = parts.select(
  "COUNTRY",
  f.split("COUNTRY", ", ").alias("COUNTRY"),
  f.posexplode(f.split("COUNTRY", ", ")).alias("pos", "COUNTRY2")
  ).select("COUNTRY2")

countries.createOrReplaceTempView("countries")
country_counts = spark.sql("SELECT COUNTRY2, COUNT(*) FROM countries GROUP BY COUNTRY2")
country_counts.show()

actors_from_rows = spark.sql("SELECT actors FROM all_movies")
actors = actors_from_rows.select(
  "actors",
  f.split("actors", ", ").alias("actors"),
  f.posexplode(f.split("actors", ", ")).alias("pos", "actors2")
  ).select("actors2")

actors.createOrReplaceTempView("actors")
actors_counts = spark.sql("SELECT actors2, COUNT(*) FROM actors GROUP BY actors2")
actors_counts.show()

NameError: name 'parts' is not defined